In [1]:
import pyaudio
import websockets
import asyncio
import base64    #보내기 전 데이터를 base64 문자열로 인코딩해야 함
import json
from openai_api_key import API_KEY_OPENAI
from assemblyai_api_key import API_KEY_ASSEMBLYAI
import openai
#conda install -c conda-forge openai

openai.api_key = API_KEY_OPENAI

In [2]:
def ask_computer(prompt):
    res = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
    )
    print(res)
    return res["choices"][0]["text"]


In [3]:
prompt = "What is your favorite color?"

ask_computer(prompt)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nMy favorite color is definitely blue!"
    }
  ],
  "created": 1673423538,
  "id": "cmpl-6XQCAKdG2y4Obc11dvr2vb8tRp4ug",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 6,
    "total_tokens": 15
  }
}


'\n\nMy favorite color is definitely blue!'

In [ ]:
#Basic의 음성 녹음 코드
FRAMES_PER_BUFFER = 3200
FORMAT = pyaudio.paInt16    
CHANNELS = 1
RATE = 8000

p = pyaudio.PyAudio()
stream = p.open(rate=RATE,
                channels=CHANNELS,
                format=FORMAT,
                input=True,
                frames_per_buffer=FRAMES_PER_BUFFER)    #OSError: No Default Input Device Available 현재 컴퓨터에 음성입력 디바이스(마이크)가 없음

In [ ]:
#websocket을 위한 url
URL = 'wss://api.assemblyai.com/v2/realtime/ws?sample_rate=16000'

In [ ]:
#데이터를 보내고 받는 함수
async def send_receive():
    print(f'Connecting websocket to url ${URL}')
    async with websockets.connect(
        URL,
        extra_headers=(("Authorization", API_KEY_ASSEMBLYAI),),
        ping_interval=5,
        ping_timeout=20
    ) as _ws:
        await asyncio.sleep(0.1)
        print("Receiving SessionBegins ...")
        session_begins = await _ws.recv()
        print(session_begins)
        print("Sending messages ...")
        async def send():
            while True:
                try:
                    data = stream.read(FRAMES_PER_BUFFER, exception_on_overflow=False)
                    data = base64.b64encode(data).decode("utf-8")
                    json_data = json.dumps({"audio_data":str(data)})
                    await _ws.send(json_data)
                except websockets.exceptions.ConnectionClosedError as e:
                    print(e)
                    assert e.code == 4008
                    break
                except Exception as e:
                    assert False, "Not a websocket 4008 error"
                await asyncio.sleep(0.01)
          
            return True
      
        async def receive():
            while True:
                try:
                    result_str = await _ws.recv()
                    result = json.loads(result_str)
                    prompt = result['text']
                    if prompt and result['message_type'] == 'FinalTranscript':
                        print("Me:", prompt)
                        answer = ask_computer(prompt)
                        print("Bot", answer)
                except websockets.exceptions.ConnectionClosedError as e:
                    print(e)
                    assert e.code == 4008
                    break
                except Exception as e:
                    assert False, "Not a websocket 4008 error"
      
        send_result, receive_result = await asyncio.gather(send(), receive())

In [ ]:
asyncio.run(send_receive())